<a href="https://colab.research.google.com/github/google/applied-machine-learning-intensive/blob/master/content/06_other_models/04_knn/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2020 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# K-Nearest-Neighbors (KNN)

The k-nearest neighbors (KNN) algorithm is a simple concept: define some distance metric between the items in your dataset, and find the K closest items. You can then use those items to predict some property of a test item. This prediction is achieved by having them somehow "vote" on it.

## KNN for Classification


In this example we will use KNN to predict whether or not a person will be diagnosed with diabetes. The dataset is the [Pima Indians Diabetes Database](https://www.kaggle.com/uciml/pima-indians-diabetes-database).

Upload your `kaggle.json` file and run the code below.

In [0]:
! chmod 600 kaggle.json && (ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle) && mv kaggle.json ~/.kaggle/ && echo 'Done'
! kaggle datasets download uciml/pima-indians-diabetes-database
! ls

Unzip the dataset.

In [0]:
! unzip pima-indians-diabetes-database.zip

And then load the dataset into a `DataFrame`.

In [0]:
import pandas as pd 

diabetes = pd.read_csv('diabetes.csv')
diabetes.sample(n=10)

Take a quick look at the data to see how many rows and columns we are dealing with.

In [0]:
diabetes.describe()

Our features are:
- Pregnancies
- Glucose
- BloodPressure
- SkinThickness
- Insulin
- BMI
- DiabetesPedigreeFunction
- Age

Our target is `Outcome`, which is currently encoded with a 1 for a positive diagnosis and 0 for a negative diagnosis.

In [0]:
print(diabetes.groupby('Outcome').size())

Notice there are several zeros in the feature columns (check the **min** values). These are likely cases where the data simply wasn't collected or stored properly. (For example, a blood pressure of 0 does not make sense.) We need to clean these up or they will have an incorrect effect on the outcome of our KNN.

In [0]:
import numpy as np
no_zero = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for column in no_zero:
  diabetes[column] = diabetes[column].replace(0, np.NaN)
  mean = int(diabetes[column].mean(skipna=True))
  diabetes[column] = diabetes[column].replace(np.NaN, mean)

diabetes.describe()

We create training and testing sets (20% for testing), remembering to separate 'Outcome' as our target value.

In [0]:
from sklearn.model_selection import train_test_split

X = diabetes.iloc[:,0:8]
y = diabetes.iloc[:,8]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Now we scale our features using StandardScaler. 

In [0]:
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test= sc_X.transform(X_test)

Finally, we use the scikit-learn KNN model. 

In [0]:
from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 14

KNN = KNeighborsClassifier(n_neighbors=n_neighbors, p=2, metric='euclidean')
KNN.fit(X_train, y_train)

y_pred = KNN.predict(X_test)

We now evaluate our model in terms of the confusion matrix, F1 score, and accuracy.

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test,y_pred)
f1 = f1_score(y_test,y_pred)
accuracy = accuracy_score(y_test,y_pred)


print('The confusion matrix is', cm)
print('The F1 score is', f1)
print('The accuracy score is', accuracy)

## K-Nearest-Neighbors for Regression

We can also use KNN for regression. In this example we'll actually build the model from scratch in order to demonstrate its simplicity.

For our model we'll use MovieLens data. MovieLens data is available in relation to the following paper:

```text
F. Maxwell Harper and Joseph A. Konstan. 2015.
The MovieLens Datasets: History and Context.
ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19.
https://doi.org/10.1145/2827872
```


In [0]:
! wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
! unzip ml-latest-small.zip

We'll use KNN to guess the rating of a movie by looking at the 10 movies that are closest to it in terms of genres and popularity.

To start, let's load up every rating in the dataset into a Pandas DataFrame:

In [0]:
import pandas as pd

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
ratings.sample(n=10)

Now we'll group everything by `movieId` and compute the mean rating for the movie.

In [0]:
import numpy as np

mean_ratings = ratings[['movieId', 'rating']].groupby('movieId').agg({'rating': ['sum', 'mean']})
mean_ratings.columns = ['rating_count', 'mean_rating']
mean_ratings.sample(n=10)

There is likely a fair amount of variance in the sum of ratings, so we'll normalize that column.

In [0]:
mean_ratings['rating_count'] = (
    (mean_ratings['rating_count'] - mean_ratings['rating_count'].min()) / 
    (mean_ratings['rating_count'].max() - mean_ratings['rating_count'].min()))

mean_ratings['rating_count'].describe()

Now let's get the genre information from the `movies.csv` file. In the genres column, we see the list of genres for each movie separated by a `'|'`. Note that a movie may have more than one genre. 

First we read the file into a DataFrame. 

In [0]:
movies = pd.read_csv('./ml-latest-small/movies.csv')
movies.sample(n=10)

In [0]:
movies.describe()

Now we split the genres column on the `'|'` and create a new DataFrame called `movies_split`. 

In [0]:
movies_split = movies.genres.str.split('|', expand=True)
movies_split.head()

We now create a list of all the unique genres that appear in this DataFrame and remove values that indicate that a genre wasn't specified.

In [0]:
genres = list(pd.unique(movies_split.values.ravel()))
genres.remove(None)
genres.remove('(no genres listed)')
genres = sorted(genres)
genres

In the movies DataFrame, we want to recode the values of the genres column to be a list of 20 0s and 1s that correspond to the values in `list` (in the order that they appear in `list`). For example, if a movie has genres 'Adventure and Children', then we would like the element in the genres column to be: \
`[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]`

In [0]:
genre_to_id = {v:i for i, v in enumerate(genres)}
genre_to_id

The function defined below iterates through a list of genres and compares the values to the elements of `genres_list`. It then returns an appropriate array of 0s and 1s as described above.

In [0]:
# Create the array of 0s and 1s based on genres.
def encode_genres(l):
  encoding = np.zeros(len(genres)).astype(int)
  for genre in l:
    if genre in genre_to_id:
      encoding[genre_to_id[genre]] = 1
  return encoding

# Test that f works as expected on an example list.
encode_genres(['Adventure', 'Children'])

We split the genres column of the movies DataFrame to be a list. We do this in preparation for applying the function, `encode_genres`.

In [0]:
movies['genres'] = movies.genres.str.split('|')
movies.sample(n=10)

We apply the function `encode_genres` to the genres column to change the elements to arrays of 0s and 1s representing the genres. We also set the index to be the `movieId`.

In [0]:
movies['genres'] = movies.genres.apply(encode_genres)
movies = movies.set_index('movieId')
movies.sample(n=10)

Now we can add the mean rating and the count of ratings to the movies. Let's first make sure that every index is accounted for.

In [0]:
np.setdiff1d(movies.index.to_numpy(), mean_ratings.index.to_numpy())

In [0]:
np.setdiff1d(mean_ratings.index.to_numpy(), movies.index.to_numpy())

It looks like we are missing some IDs from the ratings, so we need to be sure to do an inner join. We don't want to include movies with no ratings or ratings with no movies.

In [0]:
movies = movies.join(mean_ratings, how='inner')
movies.sample(n=10)

Now let's define a function that computes the "distance" between two movies based on how similar their genres are and how similar their popularity is. To make sure it works, we'll compute the distance between movie IDs `2` and `2728`:

In [0]:
from scipy import spatial

def distance(a, b):
  genre_distance = spatial.distance.euclidean(a['genres'], b['genres'])
  popularity_distance = abs(a['rating_count'] - b['rating_count'])
  return genre_distance + popularity_distance
    
distance(movies[movies.index == 2].iloc[0], movies[movies.index == 2728].iloc[0])

Remember, the higher the distance, the less similar the movies are. Let's check what movies `2` and `2728` actually are, and then let's confirm they're not all that similar:

In [0]:
print(movies[movies.index == 2].iloc[0])
print(movies[movies.index == 2728].iloc[0])

Now we just need a little code to compute the distance between some given test movie (Toy Story, in this example) and all of the movies in our dataset.

We'll find the `10` nearest neighbors utilizing a `heapq` to keep our memory usage low. Note that `heapq` pops the smallest values first, so we need to take the negative of the distance in order to remove the largest neighbors first.

In [0]:
import heapq

def k_nearest_neighbors(movie_id, K):
    distances = []
    central_movie = movies[movies.index == movie_id].iloc[0]
    for mid, movie in movies.iterrows():
        if (mid != movie_id):
            dist = distance(central_movie, movie)
            if len(distances) < K:
              heapq.heappush(distances, (-dist, mid))
            else:
              _ = heapq.heappushpop(distances, (-dist, mid))
    return [x[1] for x in distances]

avg_rating = 0.0
for id in k_nearest_neighbors(1, 10):
  neighbor = movies[movies.index == id].iloc[0]
  print(neighbor['title'], neighbor['mean_rating'])
  avg_rating += neighbor['mean_rating']

print("\nPredicted Rating: ", avg_rating/10)

How does this compare to Toy Story's actual average rating?

In [0]:
movies[movies.index == 1].iloc[0]['mean_rating']

# Exercise: `KNeighborsRegressor`

Earlier in the lab, we built a KNN regressor from scratch. Scikit-learn offers the [`KNeighborsRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html), which can perform the regression for us.

In this exercise we'll again use the MovieLens dataset to predict rating. Instead of writing your own regressor, use the [`KNeighborsRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html). You'll need to load the data, prepare it for the regressor, and then build and train your model.

Instead of building one model, build one hundred. Try using a neighbor count from `1` to `101`. Train your model using a new neighbor count each time. Keep some holdout data for testing, and calculate the root mean squared error for each neighbor count on the holdout data. Plot the RMSE data vs. the neighbor count to try to determine the optimal number of neighbors to consider for this dataset.

Explain your work. Use as many code and text blocks as you need.

**Student Solution**

In [0]:
# Your code goes here.

---